In [1]:
from general_tools.notebook import gpu_utils
GPU = 1
gpu_utils.setup_one_gpu(GPU)
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.WARN)

Picking GPU 1


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out import create_dir
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.in_out.basics import Data_Splitter, read_saved_epochs

from tf_lab.point_clouds.encoders_decoders import encoder_with_convs_and_symmetry_new, decoder_with_fc_only
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet
from tf_lab.data_sets.shape_net import pc_loader as snc_loader
from tf_lab.data_sets.shape_net import snc_category_to_synth_id

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
n_pc_samples = 2048
do_training = True
first_time_running = False
load_model = True
seed = 42
max_training_epochs = 2000
loss = 'emd'
z_rotate = True
experiment_name = 'no_conv_deeper_snc_rotated' + str(n_pc_samples) +  'pts_' + loss

In [5]:
top_data_dir = '/orions4-zfs/projects/optas/DATA/'
train_dir = osp.join(top_data_dir, 'OUT/iclr/nn_models/all_snc', experiment_name)
create_dir(train_dir)

'/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/all_snc/no_conv_deeper_snc_rotated2048pts_emd'

In [6]:
n_threads = 25
verbose = True
pc_versions = ['centered']
pclouds = []
for version in pc_versions:
    full_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/', version, str(n_pc_samples))
    full_file_names = [f for f in files_in_subdirs(full_pclouds_path, '.ply')]
    pclouds_v, _, _ = load_point_clouds_from_filenames(full_file_names, n_threads, snc_loader, verbose)
    pclouds.append(pclouds_v)
train_data = PointCloudDataSet(np.vstack(pclouds))
print train_data.num_examples

/orions4-zfs/projects/optas/Git_Repos/tf_lab/point_clouds/in_out.py:38: UserWarning: Point clouds with the same model name were loaded.
  warnings.warn('Point clouds with the same model name were loaded.')


57449 pclouds were loaded. They belong in 57 shape-classes.
57449


In [7]:
n_input = [n_pc_samples, 3]

encoder_args = {'n_filters': [128, 128, 256, 512],
                'filter_sizes': [1, 1, 1, 1],
                'strides': [1, 1, 1, 1],
                'b_norm': True
               }

decoder_args = {'layer_sizes': [1024, 2048, np.prod(n_input)],
                'b_norm': True
               }

In [8]:
if load_model:
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    print conf
    
else:    
    conf = Conf(
                n_input = n_input,
                training_epochs = max_training_epochs,
                batch_size = 50,
                loss = loss,
                denoising = False,
                train_dir = train_dir,
                loss_display_step = 1,
                saver_step = 10,
                learning_rate = 0.0005,
                z_rotate = z_rotate,
                encoder = encoder_with_convs_and_symmetry_new,
                encoder_args = encoder_args,
                decoder = decoder_with_fc_only,
                decoder_args = decoder_args,            
               )
    conf.allow_gpu_growth = False
    conf.experiment_name = experiment_name
    conf.save(osp.join(conf.train_dir, 'configuration'))

reset_tf_graph()
ae = PointNetAutoEncoder(experiment_name, conf)

              allow_gpu_growth: False
                    batch_size: 50
                 consistent_io: None
                         debug: False
                       decoder: decoder_with_fc_only
                  decoder_args: {'b_norm': True, 'layer_sizes': [1024, 2048, 6144]}
                       encoder: encoder_with_convs_and_symmetry_new
                  encoder_args: {'filter_sizes': [1, 1, 1, 1], 'n_filters': [128, 128, 256, 512], 'b_norm': True, 'strides': [1, 1, 1, 1]}
               experiment_name: no_conv_deeper_snc_rotated2048pts_emd
                 gauss_augment: None
                  is_denoising: False
               latent_vs_recon: 1.0
                 learning_rate: 0.0005
                          loss: emd
             loss_display_step: 1
                       n_input: [2048, 3]
                      n_output: [2048, 3]
                           n_z: None
             saver_max_to_keep: None
                    saver_step: 10
                     trai

In [ ]:
if load_model:
    saved_epochs = read_saved_epochs(conf.train_dir)
    last_epoch = saved_epochs[-1]
    ae.restore_model(conf.train_dir, last_epoch, verbose=True)
        
if load_model and do_training:
    conf.training_epochs -= last_epoch
    print conf.training_epochs

Model restored in epoch 1000.
1000


In [ ]:
if do_training:
    buf_size = 1 # flush each line
    fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)
    train_stats = ae.train(train_data, conf, log_file=fout)
    fout.close()

('Epoch:', '1001', 'training time (minutes)=', '4.0503', 'loss=', '82.911327980')
('Epoch:', '1002', 'training time (minutes)=', '4.3995', 'loss=', '83.225842007')
('Epoch:', '1003', 'training time (minutes)=', '4.4111', 'loss=', '82.921054677')
('Epoch:', '1004', 'training time (minutes)=', '4.3813', 'loss=', '82.923875812')
('Epoch:', '1005', 'training time (minutes)=', '5.1277', 'loss=', '82.719259990')
('Epoch:', '1006', 'training time (minutes)=', '5.2627', 'loss=', '83.179616662')
('Epoch:', '1007', 'training time (minutes)=', '4.7574', 'loss=', '82.908324876')
('Epoch:', '1008', 'training time (minutes)=', '5.5012', 'loss=', '83.235737242')
('Epoch:', '1009', 'training time (minutes)=', '5.3679', 'loss=', '82.842318060')
('Epoch:', '1010', 'training time (minutes)=', '4.6210', 'loss=', '82.454768699')
('Epoch:', '1011', 'training time (minutes)=', '4.6541', 'loss=', '82.888548422')
('Epoch:', '1012', 'training time (minutes)=', '4.6992', 'loss=', '82.793172600')
('Epoch:', '1013